In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter("ignore")

import itertools
import pandas as pd
import numpy as np

#for modeling
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose as sd
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoLarsCV

In [3]:
df = pd.read_csv('City_Zhvi_1bedroom.csv')
df.head()

,Unnamed: 0,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,CountyName,1996-01-31,...,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31
0,0,6181,0,New York,City,NY,NY,New York-Newark-Jersey City,Queens County,139028.0,...,606162.0,604457.0,603126.0,601368.0,600138.0,598527.0,597268.0,597019.0,596725.0,596432.0
1,1,12447,1,Los Angeles,City,CA,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,114152.0,...,531014.0,530916.0,530814.0,531586.0,533161.0,534943.0,537784.0,541161.0,546048.0,550710.0
2,2,39051,2,Houston,City,TX,TX,Houston-The Woodlands-Sugar Land,Harris County,68246.0,...,104937.0,105155.0,105502.0,105618.0,106083.0,106186.0,106299.0,106161.0,106271.0,106635.0
3,3,17426,3,Chicago,City,IL,IL,Chicago-Naperville-Elgin,Cook County,95803.0,...,238850.0,239106.0,239230.0,239070.0,238942.0,238700.0,238530.0,238580.0,239266.0,240143.0
4,4,6915,4,San Antonio,City,TX,TX,San Antonio-New Braunfels,Bexar County,55420.0,...,90717.0,91135.0,91256.0,91958.0,92577.0,93383.0,93500.0,93963.0,94115.0,94557.0


In [18]:
count = 0
flag = 0
prices = []
for index,i in enumerate(ny.columns):
    if i == '1996-01-31':
        start = index
        flag = 1
    elif flag==1:
        prices.append(ny.iloc[:, start+count][0])
        count+=1
prices.append(ny.iloc[:, start+count][0])

In [22]:
boxes = {'Date': list(ny.columns[9:]),
         'Price': prices
        }

df = pd.DataFrame(boxes, columns= ['Date','Price'])

In [102]:
df.index.name='index'

In [18]:
# Define the p, d and q parameters to take any value between 0 and 2
p = d = q = range(0,2)
# Generate all different combinations of p, d and q triplets
pdq = list(itertools.product(p,d,q))
# Generate all different combinations of seasonal p, d and q triplets
pdqs = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

In [103]:
df.head()

,Date,Price
index,,
0,1996-01-31,139028.0
1,1996-02-29,138821.0
2,1996-03-31,138699.0
3,1996-04-30,138671.0
4,1996-05-31,138885.0


In [23]:
def get_index(str):
    for index,i in enumerate(df['Date']):
        if i == str:
            return index

In [24]:
ts_train=df[df.index[0]:( df.index[get_index('2014-09-30')])]
ts_test=df[( df.index[get_index('2014-09-30')]):df.index[-1]]

In [137]:
ts_train['Price']

index
0      139028.0
1      138821.0
2      138699.0
3      138671.0
4      138885.0
         ...   
219    474382.0
220    478467.0
221    481038.0
222    483859.0
223    487042.0
Name: Price, Length: 224, dtype: float64

In [138]:
from pmdarima.arima import auto_arima
paramsearch=auto_arima(ts_train['Price'],start_p=2,d=None,start_q=1,max_p=7,max_d=1,max_q=5,start_P=1
                  ,D=1,start_Q=1,max_P=7,max_D=3,max_Q=7,m=12, trace=True,enforce_stationarity=False)

Performing stepwise search to minimize aic
 ARIMA(2,1,1)(1,1,1)[12]             : AIC=inf, Time=1.63 sec
 ARIMA(0,1,0)(0,1,0)[12]             : AIC=4103.245, Time=0.02 sec
 ARIMA(1,1,0)(1,1,0)[12]             : AIC=4135.797, Time=0.25 sec
 ARIMA(0,1,1)(0,1,1)[12]             : AIC=4081.642, Time=0.31 sec
 ARIMA(0,1,1)(0,1,0)[12]             : AIC=4095.395, Time=0.07 sec
 ARIMA(0,1,1)(1,1,1)[12]             : AIC=inf, Time=1.16 sec
 ARIMA(0,1,1)(0,1,2)[12]             : AIC=4075.988, Time=0.80 sec
 ARIMA(0,1,1)(1,1,2)[12]             : AIC=inf, Time=2.91 sec
 ARIMA(0,1,1)(0,1,3)[12]             : AIC=4074.573, Time=1.89 sec
 ARIMA(0,1,1)(1,1,3)[12]             : AIC=inf, Time=5.58 sec
 ARIMA(0,1,1)(0,1,4)[12]             : AIC=4068.478, Time=6.33 sec
 ARIMA(0,1,1)(1,1,4)[12]             : AIC=inf, Time=8.93 sec
 ARIMA(0,1,1)(0,1,5)[12]             : AIC=inf, Time=13.86 sec
 ARIMA(0,1,1)(1,1,5)[12]             : AIC=inf, Time=30.69 sec
 ARIMA(0,1,0)(0,1,4)[12]             : AIC=4082.715,

In [25]:
import statsmodels.api as sm
y_hat_avg = ts_test['Price'].copy()
fit1 = sm.tsa.statespace.SARIMAX(ts_train['Price'], order=(1,1,1),seasonal_order=(0,0,0,0)).fit()
a = fit1.predict(start=ts_test.index[0],end=ts_test.index[-1])

In [28]:
a

224    488390.705510
225    489665.377074
226    490870.078605
227    492008.650942
228    493084.724087
           ...      
285    510870.185802
286    510910.900328
287    510949.379928
288    510985.747282
289    511020.118338
Name: predicted_mean, Length: 66, dtype: float64